In [1]:
%matplotlib widget
import time
import os
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import sys
import os

import hcipy

import psi as psi
import instruments
from psiSensor import PsiSensor

/Users/orban/github/hcipy/hcipy/__init__.py
0.4.1.dev137+gc1ea9e5


# Basic one-iteration

In [2]:
config_file = 'config/config_metis_compass.py'
psi_sensor = PsiSensor(config_file)


PSI - [Notice] :  Loading and checking configuration


In [3]:
psi_sensor.setup()

PSI - [Notice] :  Initialize the instrument object & building the optical model
CompassInstrument - [Notice] :  Building a simple imager in HCIPy
PSI - [Notice] :  Building modal basis for projectiong/filtering of the NCPA map
PSI - [Notice] :  Results will be stored in /Users/orban/Projects/METIS/4.PSI/psi_results/orban_config_metis_compass_2022-05-13T15:09:16/


In [4]:
psi_sensor.next(display=True)

Total execution time: 278 ms


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: type object 'FigureCanvasBase' has no attribute 'start_event_loop_default'

# Parameters

In [14]:
print('PSI correction rate = {0}Hz'.format(psi_sensor.cfg.params.psi_framerate))
print('SCI DIT = {0}s'.format(psi_sensor.cfg.params.dit))
print('AO framerate = {0}Hz'.format(psi_sensor.cfg.params.ao_framerate))
print('AO telemetry decimation = {0}'.format(psi_sensor.cfg.params.ao_frame_decimation))

PSI correction rate = 1Hz
SCI DIT = 0.1s
AO framerate = 1000Hz
AO telemetry decimation = 10


In [16]:
nbOfPastSeconds = 1 / psi_sensor.cfg.params.psi_framerate
wfs_telemetry_buffer = psi_sensor.inst.grabWfsTelemetry(nbOfPastSeconds)
sci_images_buffer = psi_sensor.inst.grabScienceImages(nbOfPastSeconds)

print(wfs_telemetry_buffer.shape)
print(sci_images_buffer.shape)

(100, 256, 256)
(10, 80, 80)


In [13]:
plt.figure()
plt.imshow(wfs_telemetry_buffer[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Photometry

In [2]:
config_file = 'config/config_metis_compass.py'
psi_sensor = PsiSensor(config_file)
psi_sensor.setup()

PSI - [Notice] :  Loading and checking configuration
PSI - [Notice] :  Initialize the instrument object & building the optical model
CompassInstrument - [Notice] :  Building a simple imager in HCIPy
PSI - [Notice] :  Building modal basis for projectiong/filtering of the NCPA map
PSI - [Notice] :  Results will be stored in /Users/orban/Projects/METIS/4.PSI/psi_results/orban_config_metis_compass_2022-05-13T16:36:12/


In [11]:
def _propagateSpeckleFields(wfs_telemetry_buffer):
    nf, nx, ny = wfs_telemetry_buffer.shape
    wfs_telemetry_buffer_1d = wfs_telemetry_buffer.reshape((nf, nx*ny))
    wfs_wavefront_hcipy = hcipy.Field(wfs_telemetry_buffer_1d, psi_sensor.inst.pupilGrid)
    Efield = hcipy.Wavefront(psi_sensor.inst.aperture * np.exp(1j * wfs_wavefront_hcipy) \
                            -  psi_sensor._diffraction_component)
    Efield.total_power = psi_sensor.nbOfPhotons * nf # * nf

#     Efield_perfect = hcipy.Wavefront(psi_sensor._diffraction_component)
#     Efield_perfect.total_power = psi_sensor.nbOfPhotons / nf
    #
    speckle_fields = psi_sensor.inst.optical_model(Efield)
#     speckle_fields.total_power = psi_sensor.nbOfPhotons * nf
#     speckle_fields_perfect = psi_sensor.inst.optical_model(Efield_perfect)#.electric_field
    #
    # speckle_fields = speckle_fields - speckle_fields_perfect
#     return (speckle_fields.electric_field - speckle_fields_perfect.electric_field) * np.sqrt(speckle_fields.grid.weights)
    return (speckle_fields.electric_field) * np.sqrt(speckle_fields.grid.weights)

In [19]:
nbOfSeconds = 1/psi_sensor.cfg.params.psi_framerate
psi_sensor.inst.phase_ncpa = 0
psi_sensor.inst.include_residual_turbulence=True
psi_sensor.inst.toto_scaling = 1
# plt.figure()
# plt.imshow(np.log10(psi_sensor.inst._grabOneScienceImage()))



wfs_telemetry_buffer = psi_sensor.inst.grabWfsTelemetry(nbOfSeconds)
science_images_buffer = psi_sensor.inst.grabScienceImages(nbOfSeconds)

telemetry_indexing = psi_sensor.inst.synchronizeBuffers(wfs_telemetry_buffer,
                                                      science_images_buffer)
    
# speckle_fields = psi_sensor._propagateSpeckleFields(wfs_telemetry_buffer)
# speckle_fields = _propagateSpeckleFields(wfs_telemetry_buffer)


Efield_perfect = hcipy.Wavefront(psi_sensor._diffraction_component)
Efield_perfect.total_power = psi_sensor.nbOfPhotons
tmp = psi_sensor.inst.optical_model(Efield_perfect)#.electric_field
speckle_fields_perfect = tmp.electric_field * np.sqrt(tmp.grid.weights)

# Slicing : one speckle field per science image
for i in range(len(telemetry_indexing)):
    _s, _e = telemetry_indexing[i]
#     speckle_fields = psi_sensor._propagateSpeckleFields(wfs_telemetry_buffer[_s:_e])
    speckle_fields = _propagateSpeckleFields(wfs_telemetry_buffer[_s:_e])

#     speckle_fields -= speckle_fields_perfect
    average_speckle_field = speckle_fields.mean(axis=0) #[_s:_e,:].mean(axis=0)

    psi_sensor._speckle_field_t_psi[i] = average_speckle_field #- speckle_fields_perfect# * 100
    psi_sensor._image_t_psi[i] = science_images_buffer[i].ravel()

# TODO check if this conversion to hcipy.Field is needed
psi_sensor._speckle_field_t_psi = hcipy.Field(psi_sensor._speckle_field_t_psi,
                                        psi_sensor.inst.focalGrid)
psi_sensor._image_t_psi = hcipy.Field(psi_sensor._image_t_psi,
                                psi_sensor.inst.focalGrid)

In [17]:
speckle_fields = psi_sensor._propagateSpeckleFields(wfs_telemetry_buffer[0:10])
speckle_fields[0, 0]

(47.645445933775136-5.268867788036754j)

In [28]:
idx = 1
psf = psi_sensor._image_t_psi[idx]

r_psf = np.abs(psi_sensor._speckle_field_t_psi[idx])**2 #* 1/16
psf_perfect = np.abs(speckle_fields_perfect)**2
print(np.sum(psf) / np.sum(r_psf))
print(np.sum(psf) / np.sum(psf_perfect))

plt.figure()
plt.subplot(131)
hcipy.imshow_field(np.sqrt(r_psf  / psf.max()))
# plt.imshow(np.sqrt(np.real(psi_sensor._speckle_field_t_psi[idx]).shaped  / psf.max()))

plt.subplot(132)
# plt.imshow((np.abs(psf.shaped - 0.97*psf_perfect.shaped) / psf.max()), vmax=3e-4)
plt.imshow((np.abs(psf.shaped - 0.97*psf_perfect.shaped) / psf.max()), vmax=3e-4)


plt.subplot(133)
plt.imshow(np.sqrt((psf_perfect.shaped) / psf.max()))

    

35.83089110616331
0.9764721482301324


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# check apertures

In [20]:
%matplotlib widget
from configParser import loadConfiguration
from instruments import CompassSimInstrument
import psi
config_file = '/Users/orban/Projects/METIS/4.PSI/psi_github/config/config_metis_compass.py'
cfg = loadConfiguration(config_file)
inst = CompassSimInstrument(cfg.params)
inst.build_optical_model()

CompassInstrument - [Notice] :  Building a Classical Vortex Coronagraph optical model in HCIPy


In [46]:
conf = cfg.params
self = inst
pupil_aperture = psi.make_COMPASS_aperture(conf.f_aperture,
                                            npupil=self._size_pupil_grid,
                                            rot90=True)(self.pupilGrid)
lyot_aperture = psi.make_COMPASS_aperture(conf.f_lyot_stop,
                                                            npupil=self._size_pupil_grid,
                                                            rot90=True)(self.pupilGrid)


ncpa_file = self._prefix_ncpa + str(self._ncpa_index) + '.fits'
mask_pupil = fits.getdata(self._input_folder_ncpa + ncpa_file)
mask_pupil[mask_pupil != 0 ]=1
# mask_pupil= np.rot90(mask_pupil)
# mask_pupil = psi.psi_utils.resize_img(mask_pupil, self._size_pupil_grid)


size_ = self._size_pupil_grid
mask_pupil = psi.psi_utils.process_screen(mask_pupil, size_+6, pupil_aperture, rotate=True, ncpa_=True)
mask_pupil = psi.psi_utils.crop_img(mask_pupil, (size_, size_))
    
    


In [52]:
plt.figure()
plt.subplot(121)
plt.imshow(mask_pupil) #fits.getdata(self._input_folder_ncpa + ncpa_file))
plt.subplot(122)
plt.imshow(inst.phase_ncpa.shaped + mask_pupil + pupil_aperture.shaped)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
plt.figure()
plt.imshow(inst.phase_residual.reshape((256, 256)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
plt.figure()
# plt.imshow(pupil_aperture.shaped + lyot_aperture.shaped)
plt.subplot(121)
plt.imshow(pupil_aperture.shaped + lyot_aperture.shaped)
plt.subplot(122)
plt.imshow(pupil_aperture.shaped * mask_pupil+ lyot_aperture.shaped)

plt.figure()
plt.imshow(fits.getdata(self._input_folder_ncpa + ncpa_file))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …